Import all packages here

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW
from torch.nn import CTCLoss
from tqdm import tqdm

from dataclasses import dataclass
from typing import Dict, List, Union

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Trainer, TrainingArguments, Wav2Vec2CTCTokenizer, EarlyStoppingCallback

import librosa
from librosa.effects import trim
import librosa.display

from IPython.display import Audio

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import pydub as pyd

import jiwer as jw

import pandas as pd 
import numpy as np 

import os

import matplotlib.pyplot as plt

import re


c:\FINAL CLEFT PALATE\training-model-asr\asr-thesis-2024\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\FINAL CLEFT PALATE\training-model-asr\asr-thesis-2024\venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Mounting Gdrive for the datasets

Setting up the Dataset

In [ ]:
dataset_path = "dataset/"
metadata = "dataset.csv"

audio_directory = "dataset/"

# Create a dataframe for the transcript
dataframe = pd.read_csv(metadata)

# Preprocess transcript
def preprocess_text(text):
    text = text.upper()  # Convert text to uppercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove non-alphanumeric characters (except spaces)
    return text

dataframe['clean_transcript'] = dataframe['Transcription'].apply(preprocess_text)

print(dataframe.head())

  File_Path Speaker         Transcription Session      clean_transcript
0     03M_1     03M  1 2 3 4 5 6 7 8 9 10       1  1 2 3 4 5 6 7 8 9 10
1     03M_2     03M                   ata       2                   ATA
2     03M_3     03M                   ana       3                   ANA
3     03M_4     03M                   ara       4                   ARA
4     03M_5     03M                  atha       5                  ATHA


Data Pre-Processing

In [3]:


# Function to preprocess audio and connect to transcripts
def combine_audio_with_transcript(directory, dataframe):
    audio_data = []    
     # Iterate over each row in the DataFrame
    for index, row in dataframe.iterrows():
        file_name = row['File_Path']  # Get the file name from the CSV (without .wav)
        transcript = row['clean_transcript']  # Get the transcript
        
        # Construct the full file path by combining directory and file name with .wav extension
        file_path = os.path.join(directory, f"{file_name}.wav")
        
        # Check if the file exists in the audio directory
        if os.path.exists(file_path):
            try:
                # Append the processed data along with the transcript
                audio_data.append({
                    "file_path": file_path,
                    "transcript": transcript,
                })
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    return audio_data

# Preprocess audio files and connect to transcripts
audio_data_with_transcripts = combine_audio_with_transcript(audio_directory, dataframe)


Convert Dataset into Pytorch Dataset

In [4]:
audio_with_transcript_dataframe = pd.DataFrame(audio_data_with_transcripts)

for index, row in audio_with_transcript_dataframe.iterrows():
    file_path = row['file_path']
    try:
        # Load the audio file
        audio, sr = librosa.load(file_path, sr=None)  # sr=None to preserve the original sample rate
        
        # Display basic information about the audio
        print(f"Audio file {file_path} loaded successfully")
        print(f"Audio length: {len(audio)} samples")
        print(f"Sample rate: {sr} Hz\n")
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

Audio file dataset/03M_1.wav loaded successfully
Audio length: 717953 samples
Sample rate: 48000 Hz

Audio file dataset/03M_2.wav loaded successfully
Audio length: 561018 samples
Sample rate: 48000 Hz

Audio file dataset/03M_3.wav loaded successfully
Audio length: 697276 samples
Sample rate: 48000 Hz

Audio file dataset/03M_4.wav loaded successfully
Audio length: 643847 samples
Sample rate: 48000 Hz

Audio file dataset/03M_5.wav loaded successfully
Audio length: 864068 samples
Sample rate: 48000 Hz

Audio file dataset/03M_6.wav loaded successfully
Audio length: 728750 samples
Sample rate: 48000 Hz

Audio file dataset/03M_7.wav loaded successfully
Audio length: 715758 samples
Sample rate: 48000 Hz

Audio file dataset/03M_8.wav loaded successfully
Audio length: 484327 samples
Sample rate: 48000 Hz

Audio file dataset/03M_9.wav loaded successfully
Audio length: 453342 samples
Sample rate: 48000 Hz

Audio file dataset/03M_10.wav loaded successfully
Audio length: 483563 samples
Sample rate:

Build The Model

In [5]:
# Initialize the processor
asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Initialize the model
asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
asr_tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-960h")


print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU Name:", torch.cuda.get_device_name(0))


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CUDA available: True
CUDA version: 12.1
GPU Name: NVIDIA GeForce RTX 3060 Laptop GPU


In [6]:
sampling_rate=16000

def prepare_dataset(batch):
    """
    Prepare dataset with correct data types.
    """
    # Load the audio file
    audio, sr = librosa.load(batch['file_path'], sr=sampling_rate, mono=True)
    
    # Trim silence from the audio
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=20)
    
    # Process audio
    audio_features = asr_processor(
        trimmed_audio, 
        sampling_rate=sampling_rate, 
        padding=False,
        return_tensors=None
    ).input_values[0]
    
    # Ensure audio features are float32
    audio_features = audio_features.astype(np.float32)
    
    # Process the transcript
    with asr_tokenizer.as_target_tokenizer():
        labels = asr_tokenizer(
            batch['transcript'],
            padding=False,
            return_tensors=None
        ).input_ids
    
    return {
        "input_values": audio_features,
        "labels": labels
    }

def custom_data_collator(batch):
    """
    Custom collator with correct data types.
    """
    # Get max length in the batch
    max_audio_length = max(len(x["input_values"]) for x in batch)
    max_label_length = max(len(x["labels"]) for x in batch)
    
    batch_audio = []
    batch_labels = []
    
    for sample in batch:
        # Pad audio
        audio_length = len(sample["input_values"])
        padded_audio = np.pad(
            sample["input_values"],
            (0, max_audio_length - audio_length),
            mode='constant',
            constant_values=0
        )
        batch_audio.append(padded_audio)
        
        # Pad labels
        label_length = len(sample["labels"])
        padded_labels = np.pad(
            sample["labels"],
            (0, max_label_length - label_length),
            mode='constant',
            constant_values=-100
        )
        batch_labels.append(padded_labels)
    
    # Convert to tensors with correct dtypes
    batch_audio = torch.tensor(batch_audio, dtype=torch.float32)
    batch_labels = torch.tensor(batch_labels, dtype=torch.long)
    
    return {
        "input_values": batch_audio,
        "labels": batch_labels
    }

In [7]:
dataset = Dataset.from_pandas(audio_with_transcript_dataframe)
processed_dataset = dataset.map(prepare_dataset, remove_columns=["file_path", "transcript"])

# Check if dataset is good
processed_dataframe = processed_dataset.to_pandas()
print(processed_dataframe.head())

Map:   0%|          | 0/2054 [00:00<?, ? examples/s]

c:\FINAL CLEFT PALATE\training-model-asr\asr-thesis-2024\venv\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2054/2054 [00:19<00:00, 103.78 examples/s]


                                        input_values  \
0  [-0.08597469, -0.06963344, -0.093899734, -0.04...   
1  [-0.013693145, 0.00078774075, -0.010232744, -0...   
2  [-0.22992292, -0.20895448, -0.16145737, -0.156...   
3  [-0.025115486, -0.03066014, -0.0062442967, -0....   
4  [-0.10101376, -0.12410567, -0.19890745, -0.223...   

                                              labels  
0  [3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, ...  
1                                          [7, 6, 7]  
2                                          [7, 9, 7]  
3                                         [7, 13, 7]  
4                                      [7, 6, 11, 7]  


In [8]:
split_dataset = processed_dataset.train_test_split(test_size=0.3, seed=42)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Print dataset details
print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 1437
Validation dataset size: 617


In [9]:
train_samples = len(train_dataset)
batch_size = 4  # Your batch size
n_gpu = 1  # Assuming single GPU
gradient_accumulation = 1  # Default value

steps_per_epoch = train_samples // (batch_size * n_gpu * gradient_accumulation)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    dataloader_num_workers=0,
    fp16=True,
    num_train_epochs=50,
    logging_steps=steps_per_epoch,
    eval_steps=steps_per_epoch,
    save_steps=steps_per_epoch,
    learning_rate=1e-7,
    lr_scheduler_type="linear",  # Options: 'linear', 'cosine', 'constant', etc.

    load_best_model_at_end=True,  # Enable saving the best model
    metric_for_best_model="eval_loss",  # Replace with your desired metric
    greater_is_better=False,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
)

trainer = Trainer(
    model=asr_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=custom_data_collator,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
    ],
)


c:\FINAL CLEFT PALATE\training-model-asr\asr-thesis-2024\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training the Model

In [10]:
trainer.train()

  0%|          | 0/18000 [00:00<?, ?it/s]C:\Users\CARLOS FIDEL ROSALES\AppData\Local\Temp\ipykernel_5116\4136957558.py:70: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  batch_audio = torch.tensor(batch_audio, dtype=torch.float32)
  2%|▏         | 360/18000 [01:42<59:23,  4.95it/s]  

{'loss': 328.1096, 'grad_norm': 8283.017578125, 'learning_rate': 9.812777777777778e-08, 'epoch': 1.0}


                                                   
  2%|▏         | 360/18000 [02:49<59:23,  4.95it/s]

{'eval_loss': 882.4044799804688, 'eval_runtime': 66.433, 'eval_samples_per_second': 9.288, 'eval_steps_per_second': 1.174, 'epoch': 1.0}


  4%|▍         | 719/18000 [04:25<1:06:03,  4.36it/s]  

...

                                                       
100%|██████████| 18000/18000 [1:42:04<00:00,  3.73it/s]

{'eval_loss': 376.72137451171875, 'eval_runtime': 27.0789, 'eval_samples_per_second': 22.785, 'eval_steps_per_second': 2.88, 'epoch': 50.0}


100%|██████████| 18000/18000 [1:42:07<00:00,  2.94it/s]

{'train_runtime': 6127.719, 'train_samples_per_second': 11.725, 'train_steps_per_second': 2.937, 'train_loss': 203.2487152777778, 'epoch': 50.0}


TrainOutput(global_step=18000, training_loss=203.2487152777778, metrics={'train_runtime': 6127.719, 'train_samples_per_second': 11.725, 'train_steps_per_second': 2.937, 'total_flos': 3.4487335393815025e+18, 'train_loss': 203.2487152777778, 'epoch': 50.0})

Training Results

Visualization of Results using MatPlotLib

Save the model

More evaluation 